# import dependencies

In [143]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import cv2 as cv
import os
# from tqdm import tqdm
import string
import skimage as ski
import sklearn as skl

In [144]:
data_dir = "dataset/asl_alphabet_train/asl_alphabet_train"
categories = list(string.ascii_uppercase)

In [145]:
print(categories)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [146]:
train_images = []
test_images = []
train_lables = []
test_lables = []

In [147]:
train_size = 400
test_size = 50
img_size = 124

In [148]:
from tqdm import tqdm
for category in tqdm(categories, desc="Loading Images"): 
    path = os.path.join(data_dir, category);
    label = ord(category) - ord('A')
    i = 0
    for img in os.listdir(path) :
        try :
            img_array=ski.io.imread(os.path.join(path,img), as_gray=True)
            new_array=ski.transform.resize(img_array,(img_size,img_size))
            if i < train_size :
                train_images.append(new_array)
                train_lables.append(label)
            else :
                test_images.append(new_array)
                test_lables.append(label)
        except Exception as e:
                print(e)
                break
        i += 1
        if i >= train_size + test_size :
            break

Loading Images: 100%|███████████████████████████| 26/26 [00:37<00:00,  1.44s/it]


In [149]:
def extract_hog_features(images):
    hog_features = []
    for image in tqdm(images, desc="Extracting features") :
        features, hog_image = ski.feature.hog(image,
                                              pixels_per_cell=(8, 8),
                                              cells_per_block=(2, 2),
                                              block_norm='L1',
                                              visualize=True,
                                              transform_sqrt=True,
                                              feature_vector=True,
                                              channel_axis=None)
        hog_features.append(features)
    return np.array(hog_features)

In [150]:
X_train = extract_hog_features(train_images)

Extracting features: 100%|████████████████| 10400/10400 [12:05<00:00, 14.33it/s]


In [151]:
X_test = extract_hog_features(test_images)

Extracting features: 100%|██████████████████| 1300/1300 [01:33<00:00, 13.91it/s]


In [152]:
y_train = train_lables
y_test = test_lables

In [153]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm

In [154]:
svc = skl.svm.SVC(kernel='linear')
svc.fit(X_train, y_train)

SVC(kernel='linear')

In [156]:
y_pred = svc.predict(X_test)

In [160]:
print(skl.metrics.accuracy_score(y_test, y_pred))

0.9346153846153846
